# Testando o spark

Este notebook testa o funcionamento do [Apache Spark](https://spark.apache.org/) no projeto.

Aqui, iremos instalar o [pyspark](https://spark.apache.org/docs/latest/api/python/) e utiliza-lo para ler os arquivos da nossa base de dados.

Vale ressaltar que para executar esse notebook, é necessário ter baixado a base de dados que será utilizada no projeto: [Anime Dataset](https://www.kaggle.com/datasets/dbdmobile/myanimelist-dataset). Após baixar, é preciso criar uma pasta chamada `dataset`, e dentro dela colocar os arquivos `.csv` contidos na pasta compactada.


## Instalando o Spark (pyspark)


A instalação do pyspark levou em conta a seguinte [página da documentação](https://spark.apache.org/docs/latest/api/python/getting_started/install.html#using-pypi).

In [1]:
%pip install pyspark

Note: you may need to restart the kernel to use updated packages.


## Importações

In [1]:
import os
import sys
from pyspark.sql import SparkSession

## Constantes

In [2]:
DATASET_PATH = '../../dataset'

ANIME_DATASET_PATH = os.path.join(DATASET_PATH, 'anime-dataset-2023.csv')
ANIME_FILTERED_PATH = os.path.join(DATASET_PATH, 'anime-filtered.csv')
FINAL_ANIME_DATASET_PATH = os.path.join(DATASET_PATH, 'final_animedataset.csv')
USER_FILTERED_PATH = os.path.join(DATASET_PATH, 'user-filtered.csv')
USER_DETAILS_PATH = os.path.join(DATASET_PATH, 'users-details-2023.csv')
USER_SCORES_PATH = os.path.join(DATASET_PATH, 'users-score-2023.csv')

## Código para resolver um problema de versão encontrada pelo PySpark

A resolução do erro foi encontrada em uma resposta no [StackOverflow](https://stackoverflow.com/a/65010346)

In [3]:
os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

## Inicializando o spark

### Instalando e configurando hadoop

Para adicionar os "conectors" de banco de dados para o spark, que serão necessários para integrar o Redis e o Neo4j com o spark, é necessário instalar e configurar o hadoop. Para fazer isso, siga o tutorial abaixo:

1- Clone o repositório [winutils](https://github.com/cdarlint/winutils)

2- Adicione a seguinte variável de ambiente: `HADOOP_HOME=<CAMINHO-ATÉ-HADOOP>/hadoop-<VERSÃO>`

O tutorial utilizado como base pode ser [acessado por aqui](https://medium.com/@enriquecatala/java-io-filenotfoundexception-hadoop-home-and-hadoop-home-dir-are-unset-4004d5e05f67)

OBS: a versão utilizada pelo projeto foi a 3.3.5

### Instalando e configurando o conector Neo4j para spark

Para isso, é necessário instalar o conector spark-neo4j. Baixe o conector ([disponível aqui](https://neo4j.com/product/connectors/apache-spark-connector/)) e extraia o arquivo `neo4j-connector-apache-spark_2.12-5.0.1_for_spark_3.jar` no repositório principal do projeto, dentro de uma pasta chamada `spark-neo4j`.

A documentação para o uso do neo4j + spark está disponível [aqui](https://neo4j.com/docs/spark/current/python/).

### Instalando e configurando o conector Redis para spark

Antes, para instalar o spark-redis, é necessário ter instalado o [Maven](https://maven.apache.org/). Para fazer isso, siga o paso a passo do [vídeo disponível aqui](https://www.youtube.com/watch?v=-ucX5w8Zm8s).

Além disso, é preciso possuir o JDK8 em sua máquina, e com o Maven configurado para usar essa versão do JDK. Para isso, baixe o JDK8, [disponível aqui](https://www.oracle.com/br/java/technologies/javase/javase8-archive-downloads.html), e adicione uma variável de ambiente `JAVA_HOME=<caminho-para-JDK8>`.

Depois disso tudo, é preciso baixar e compilar o conector [spark-redis](https://redis.com/blog/getting-started-redis-apache-spark-python/). Isso pode ser feito seguindo os passos a seguir:

- git clone https://github.com/RedisLabs/spark-redis.git
- cd spark-redis
- mvn clean package -DskipTests

OBS: se ocorrer o erro: `duplicate class: com.redislabs.provider.redis.util.BenchmarkTest`, abra a pasta `spark-redis/src` e apague a pasta `test`. Esse problema foi relatado e resolvido dessa forma em uma [issue do GitHub do próprio spark-redis](https://github.com/RedisLabs/spark-redis/issues/119)

Documentação do conector [disponível aqui](https://github.com/RedisLabs/spark-redis).

### Configurando inicialização dos "conectors" spark

Para "dizer" ao spark qual conector utilizar, foi utilizado a configuração de inicialização `spark.jars`, do método `config`, como descrito na [documentação](https://spark.apache.org/docs/latest/configuration.html)

In [4]:
spark = SparkSession.\
        builder.\
        config("spark.jars", "../../spark-neo4j/neo4j-connector-apache-spark_2.12-5.0.1_for_spark_3.jar,../../spark-redis/target/spark-redis_2.12-3.1.0-SNAPSHOT-jar-with-dependencies.jar").\
        getOrCreate()

## Lendo a base de dados

Os arquivos da base de dados possuem alguns campos que são _multilines_, ou seja, que o próprio valor do campo possui quebras de linha. O pyspark, por padrão, ao ler quebra de linhas as considera como um novo registro, mas não queremos isso. 

Para resolver isso, algumas configurações são passadas ao método de leitura de csv: multiline=True e escape='"'

Essa solução foi encontrada no [Stack Overflow](https://stackoverflow.com/a/50190622)

### Lendo o arquivo _anime-dataset-2023.csv_

In [6]:
df = spark.read.csv(ANIME_DATASET_PATH, inferSchema=True, header=True, escape='"', multiLine=True)

In [7]:
df.show()

+--------+--------------------+--------------------+------------------------------+-----+--------------------+--------------------+-----+--------+--------------------+-----------+----------------+--------------------+--------------------+----------------+-----------+-------------+--------------------+------+----------+---------+---------+-------+--------------------+
|anime_id|                Name|        English name|                    Other name|Score|              Genres|            Synopsis| Type|Episodes|               Aired|  Premiered|          Status|           Producers|           Licensors|         Studios|     Source|     Duration|              Rating|  Rank|Popularity|Favorites|Scored By|Members|           Image URL|
+--------+--------------------+--------------------+------------------------------+-----+--------------------+--------------------+-----+--------+--------------------+-----------+----------------+--------------------+--------------------+----------------+-----

In [8]:
df.select().count()

24905

### Lendo o arquivo _anime-filtered.csv_

In [9]:
df_anime_filtred = spark.read.csv(ANIME_FILTERED_PATH, inferSchema=True, header=True, escape='"', multiLine=True)

In [10]:
df_anime_filtred.show()

+--------+--------------------+-----+--------------------+--------------------+------------------------------+--------------------+-----+--------+--------------------+-----------+--------------------+--------------------+----------------+-----------+---------------+--------------------+------+----------+-------+---------+--------+---------+-------+-------+
|anime_id|                Name|Score|              Genres|        English name|                 Japanese name|           sypnopsis| Type|Episodes|               Aired|  Premiered|           Producers|           Licensors|         Studios|     Source|       Duration|              Rating|Ranked|Popularity|Members|Favorites|Watching|Completed|On-Hold|Dropped|
+--------+--------------------+-----+--------------------+--------------------+------------------------------+--------------------+-----+--------+--------------------+-----------+--------------------+--------------------+----------------+-----------+---------------+----------------

In [11]:
df_anime_filtred.count()

14952

### Lendo o arquivo _final_animedataset.csv_

In [12]:
df_final_anime = spark.read.csv(FINAL_ANIME_DATASET_PATH, inferSchema=True, header=True, escape='"', multiLine=True)

In [13]:
df_final_anime.show()

+--------+--------+--------+-------+------+--------------------+----+------------+-----+---------+------+----------+--------------------+
|username|anime_id|my_score|user_id|gender|               title|type|      source|score|scored_by|  rank|popularity|               genre|
+--------+--------+--------+-------+------+--------------------+----+------------+-----+---------+------+----------+--------------------+
|karthiga|      21|       9|2255153|Female|           One Piece|  TV|       Manga| 8.54|   423868|  91.0|        35|Action, Adventure...|
|karthiga|      59|       7|2255153|Female|             Chobits|  TV|       Manga| 7.53|   175388|1546.0|       188|Sci-Fi, Comedy, D...|
|karthiga|      74|       7|2255153|Female|        Gakuen Alice|  TV|       Manga| 7.77|    33244| 941.0|      1291|Comedy, School, S...|
|karthiga|     120|       7|2255153|Female|       Fruits Basket|  TV|       Manga| 7.77|   167968| 939.0|       222|Slice of Life, Co...|
|karthiga|     178|       7|225515

In [14]:
df_final_anime.count()

35305695

### Lendo o arquivo _user-filtered.csv_

In [15]:
df_user_filtred = spark.read.csv(USER_FILTERED_PATH, inferSchema=True, header=True, escape='"', multiLine=True)

In [16]:
df_user_filtred.show()

+-------+--------+------+
|user_id|anime_id|rating|
+-------+--------+------+
|      0|      67|     9|
|      0|    6702|     7|
|      0|     242|    10|
|      0|    4898|     0|
|      0|      21|    10|
|      0|      24|     9|
|      0|    2104|     0|
|      0|    4722|     8|
|      0|    6098|     6|
|      0|    3125|     9|
|      0|     481|    10|
|      0|      68|     6|
|      0|    1689|     6|
|      0|    2913|     6|
|      0|    1250|     7|
|      0|     356|     9|
|      0|     121|     9|
|      0|     430|     9|
|      0|    1829|     7|
|      0|    1571|    10|
+-------+--------+------+
only showing top 20 rows



In [17]:
df_user_filtred.count()

109224747

### Lendo o arquivo _user-details-2023.csv_

In [18]:
df_user_details = spark.read.csv(USER_DETAILS_PATH, inferSchema=True, header=True, escape='"', multiLine=True)

In [19]:
df_user_details.show()

+------+---------------+------+-------------------+--------------------+-------------------+------------+----------+--------+---------+-------+-------+-------------+-------------+---------+----------------+
|Mal ID|       Username|Gender|           Birthday|            Location|             Joined|Days Watched|Mean Score|Watching|Completed|On Hold|Dropped|Plan to Watch|Total Entries|Rewatched|Episodes Watched|
+------+---------------+------+-------------------+--------------------+-------------------+------------+----------+--------+---------+-------+-------+-------------+-------------+---------+----------------+
|     1|          Xinil|  Male|1985-03-03 21:00:00|          California|2004-11-04 22:00:00|       142.3|      7.37|     1.0|    233.0|    8.0|   93.0|         64.0|        399.0|     60.0|          8458.0|
|     3|        Aokaado|  Male|               null|        Oslo, Norway|2004-11-10 22:00:00|        68.6|      7.34|    23.0|    137.0|   99.0|   44.0|         40.0|       

In [20]:
df_user_details.count()

731290

### Lendo o arquivo _user-score-2023.csv_

In [21]:
df_user_score = spark.read.csv(USER_SCORES_PATH, inferSchema=True, header=True, escape='"', multiLine=True)

In [22]:
df_user_score.show()

+-------+--------+--------+--------------------+------+
|user_id|Username|anime_id|         Anime Title|rating|
+-------+--------+--------+--------------------+------+
|      1|   Xinil|      21|           One Piece|     9|
|      1|   Xinil|      48|         .hack//Sign|     7|
|      1|   Xinil|     320|              A Kite|     5|
|      1|   Xinil|      49|    Aa! Megami-sama!|     8|
|      1|   Xinil|     304|Aa! Megami-sama! ...|     8|
|      1|   Xinil|     306|Abenobashi Mahou☆...|     8|
|      1|   Xinil|      53|       Ai Yori Aoshi|     7|
|      1|   Xinil|      47|               Akira|     5|
|      1|   Xinil|     591|      Amaenaide yo!!|     6|
|      1|   Xinil|      54|   Appleseed (Movie)|     7|
|      1|   Xinil|      55|         Arc the Lad|     5|
|      1|   Xinil|      56|             Avenger|     6|
|      1|   Xinil|      57|                Beck|     9|
|      1|   Xinil|     368|         Bible Black|     5|
|      1|   Xinil|      68|      Black Cat (TV)|

In [23]:
df_user_score.count()

24325191

## Conectando o spark com os bancos de dados

### Escrita

In [5]:
df = spark.createDataFrame([
    {"name": "Gregório"}
])
df.show()

+--------+
|    name|
+--------+
|Gregório|
+--------+



In [6]:
df.write \
  .format("org.neo4j.spark.DataSource") \
  .mode("Append") \
  .option("url", "bolt://localhost:7687") \
  .option("labels", ":Person") \
  .save()

### Leitura

In [7]:
df = spark.read.format("org.neo4j.spark.DataSource") \
  .option("url", "bolt://localhost:7687") \
  .option("labels", "Person") \
  .load(inferSchema=True)
df.show()

+----+--------+--------+
|<id>|<labels>|    name|
+----+--------+--------+
|   0|[Person]|Gregório|
|   1|[Person]|Gregório|
+----+--------+--------+



## Conectando Spark com o Redis

### Escrita

In [8]:
df = spark.createDataFrame([{
    'col1': "teste1",
    'col2': "teste2"
}])

In [9]:
df.write\
  .format("org.apache.spark.sql.redis")\
  .option("table", "person")\
  .save()

### Leitura

In [10]:
df = spark.read.format("org.apache.spark.sql.redis").\
     option("table", "person").\
     load(inferSchema=True)
df.show()

+------+------+
|  col1|  col2|
+------+------+
|teste1|teste2|
+------+------+

